# Audification
- Line of Interest (Circle)
- Intensity mapping
- Linear interpolation

In [ ]:
import matplotlib.pyplot as plt
import pyamapping
from IPython.display import Audio
from scipy.io.wavfile import write
from datetime import datetime
from pya import *

%matplotlib widget

In [ ]:
s = startup()

# Simulation

In [ ]:
import simulation

n = 128
sim_speed = 0.004
sim_fps = 400
duration = 6
frame_amount = duration * sim_fps

initial_state = np.array([[simulation.gaussian_x_impulse(x, y, n, offset=[-0.6, 0.0], width=0.15, height=0.15, impulse=-0.07) for x in range(n)] for y in range(n)])

potential = np.array([[simulation.parabola(x, y, n, offset=(0, 0), factor=(10000, 10000)) for x in range(n)] for y in range(n)])
# potential = np.zeros((n, n))

barrier_x = n//2 - 1
barrier_width = 2
multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-4, -2), (2, 4)]
single_slit = [(-2, 2)]
slits = double_slit

frames = simulation.sim(n, sim_fps, duration, slits, barrier_x, barrier_width, sim_speed, initial_state=initial_state, potential=potential, normalize=True)

In [ ]:
import matplotlib
plt.pcolormesh(np.abs(frames[0]) ** 2, cmap='inferno', norm=matplotlib.colors.PowerNorm(vmin=0, vmax=np.max(np.square(np.abs(frames))), gamma=0.4))
plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()

# Video

In [ ]:
import video

# save video
video_filename, anim = video.create(frames, 20, 1, frame_amount, sim_fps, slits, barrier_x, barrier_width, n)

# plt.close()

# Sonification

In [ ]:
# circle of interest
def circle(radius, center):
    radians_per_sample = 2 * np.pi * frequency / sample_rate
    radians = np.arange(sample_amount) * radians_per_sample
    x = (0.5 * n * (radius * np.sin(radians) + 1 + center[0]))
    y = (0.5 * n * (radius * np.cos(radians) + 1 + center[1]))
    return x, y

# half circle (bottom to top)
def half_circle(radius, center):
    t = np.arange(sample_amount) / sample_rate * frequency % 1
    x = (0.5 * n * (radius * np.sin(np.pi * t) + 1 + center[0]))
    y = (0.5 * n * (radius * np.cos(np.pi * t) + 1 + center[1]))
    return x, y

# line of interest
def line(start, end):
    t = np.arange(sample_amount) / sample_rate * frequency % 1
    x = start[0]+n//2 + t * (end[0] - start[0])
    y = start[1]+n//2 + t * (end[1] - start[1])
    return x, y

In [ ]:
# parameters
sample_rate = 44100
frequency = 220
sample_amount = sample_rate * duration

# choose method
do_left = True
# x, y = circle(radius=0.25, center=[0, 0])
x, y = half_circle(radius=0.6, center=[0, 0])
# x, y = line(start=[-60, 0], end=[60, 0])

In [ ]:
# Sonification
frames_indices = np.linspace(0, frame_amount - 1, sample_amount, endpoint=False)

# No interpolation
#audio = frames[frames_indices.astype(int), y.astype(int), x.astype(int)]
#audio = np.square(np.abs(audio))

def interpolate(frames, indices, x_vals, y_vals):
    # Hardcoded interpolation
    # Way faster than any other interpolation!
    t = np.array([indices, y_vals, x_vals]) % 1
    floors = np.floor([indices, y_vals, x_vals]).astype(int)
    ceils  = np.ceil ([indices, y_vals, x_vals]).astype(int)

    return ((1-t[0]) * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[floors[0], floors[1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[floors[0], floors[1], ceils [2]])))
                     +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[floors[0], ceils [1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[floors[0], ceils [1], ceils [2]]))))
             + t[0]  * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[ceils [0], floors[1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[ceils [0], floors[1], ceils [2]])))
                     +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[ceils [0], ceils [1], floors[2]]))
                                 +     t[2]  * np.square(np.abs(frames[ceils [0], ceils [1], ceils [2]])))))

audio = interpolate(frames, frames_indices, x, y)

if do_left:
    audio_left = interpolate(frames, frames_indices, np.array(n) - x, y)

In [ ]:
# create pya
pa = Asig(audio, sr=sample_rate).fade_in(0.005).fade_out(0.005).norm().stereo()
save_audio = pa

In [ ]:
if do_left:
    pb = Asig(np.array([audio_left, audio]).transpose((1, 0)), sr=sample_rate).fade_in(0.005).fade_out(0.005).norm()
    save_audio = pb

In [ ]:
# save WAV
audio_filename = f'output/sonification_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.wav'
save_audio.save_wavfile(audio_filename)
print(f"Sonification saved as {audio_filename}")

In [ ]:
# play directly
Audio(save_audio.mono().sig, rate=sample_rate)

In [ ]:
plt.figure()
plt.plot(audio)

# Combine Video & Audio

In [ ]:
# combine audio & video
video.combine(audio_filename, video_filename)